In [6]:
import csv
from torch.utils.data import Dataset
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from bs4 import BeautifulSoup
import string
import spacy
import re
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence

EMBED_DIM = 300
HIDDEN_DIM = 100

torch.manual_seed(0)

nlp = spacy.load('en_core_web_sm')

In [3]:
file = open('processed_text.txt','a')
punctuation_words = open('punct.txt','a')

punctuations = set(list(string.punctuation))


def preprocess_text(text):
    
    text = BeautifulSoup(text,'lxml').get_text().strip()
    text = text.lower()
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)
    new_text = []
    for word in text.split(' '):
        if '.' in word and len(word)>1:
            if 'http' in word:
                words = word.split('.')
                words[1] = ''.join(words[1:])
                words = [words[0],words[1]]
            else:
                words = word.split('.')
            
            for w in words:
                new_text.append(w)
        else:
            new_text.append(word)
    text = ' '.join(new_text)

    words = nlp(text)
    review =  " ".join([re.sub(r'^https?:\/\/.*[\r\n]*', '', token.text, flags=re.MULTILINE) for token in words])
    for word in review.split(' '):
        if any([punct in word for punct in punctuations]):
            punctuation_words.write(word + "\n")


    file.write(text + '\n')
    return text


preprocessed_dataset = []
train_dataset_labels = []


with open("./Train dataset.csv") as csvfile:
    csvFile = csv.reader(csvfile)
    next(csvFile)

    for line in csvFile:
        processed_text = preprocess_text(line[0])
        label = 1 if line[1] == 'positive' else 0
        train_dataset_labels.append(label)
        preprocessed_dataset.append({"text":processed_text,"label":label})

train_dataset_labels = np.array(train_dataset_labels)
file.close()
punctuation_words.close()


/Users/depressedcoder/environments/gymenv/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [ ]:
train_idx,valid_idx = train_test_split(np.arange(train_dataset_labels.shape[0]), 
    test_size=0.2,
    shuffle= True,
    stratify= train_dataset_labels,
    random_state=0
)

In [ ]:
class ReviewDataSet(Dataset):

    def __init__(self) -> None:
        super().__init__()
    
    def __len__(self):
        return 0
    
    def __getitem__(self, index):
        return None


In [ ]:
# bidirectional
# rnn_type
# use_cnn
# num_layers

class Model(nn.Module):

    def __init__(self,
            embed_dim=EMBED_DIM,hidden_dim =HIDDEN_DIM,
            rnn_type = 'gru',num_layers=1):
        
        super().__init__()
        self.embed_dim = embed_dim
        self.hidden_dim = hidden_dim
        self.rnn = None

        if rnn_type == 'gru':
            self.rnn  = nn.GRU(input_size = self.embed_dim,
                hidden_size = self.hidden_dim,
                num_layers = num_layers,
                batch_first = True,
                dropout = 0,
                bidirectional = True
            )
        

    def forward(self,x,x_len):
        pass
        



